In [32]:
print("first age where your net assets reach 0 is " + str(Calculate_Age()))
print("you can increase your expenses by " + str(Max_Expense()))
print("you can reduce your income by " + str(Min_Income()))
print("your earliest retirement age is " + str(Calculate_Retirement()))
print("you lowest investment rate is " + str(Calculate_Investment_Rate()))
print("you lowest investment allocation is " + str(Calculate_Investment_Percentage()))

first age where your net assets reach 0 is 23
you can increase your expenses by 15%
you can reduce your income by 16%
your earliest retirement age is 12
you lowest investment rate is 4%
you lowest investment allocation is 10%


<ipython-input-17-3e1b4f76c0c6>:53: DeprecationWarning: numpy.pmt is deprecated and will be removed from NumPy 1.20. Use numpy_financial.pmt instead (https://pypi.org/project/numpy-financial/).
  payment = np.pmt(house_interest_rate,house_loan_years,-remaining_value)


## General Data

In [23]:
import numpy as np

# Serializers
# Personal Inputs
starting_age = 3
starting_savings = 5000
retirement_age = 12
life_expectancy = 20
percentage_investment = 0.3

# Macro Rates
inflation_rate = 0.0252
housing_price_index = 0.0596
salary_rate = 0.0420
investment_rate = 0.1
bank_interest_rate = 0.01

# Income Inputs
# Each income input needs its own 5 variables, amount, starting, ending, rate and bonus
total_monthly_income = [110,4,19,0.042,2,55,4,19,0.042,2,55,4,19,0.042,2]

# Expense Inputs
# Each expense input needs its own 4 variables, amount, starting, ending and rate
total_monthly_expense =[50,4,20,0.0252,50,4,20,0.0252]

# House Inputs
house_value = 2500
house_purchase_age = 4
house_loan_years = 15
house_downpayment_percentage = 0.1
house_interest_rate = 0.026

# This is needed for computation
# Creating the Age Column
Final_Age = life_expectancy+20 # We will only project 20 years past life expectancy 
Age_List = list(range(starting_age,Final_Age+1)) 

## Income

In [2]:
def income(ra=retirement_age):
   
    # Creating lists for different income streams
    amount = []
    starting = []
    ending = []
    rate = []
    bonus = []
    
    # Putting FE values into the respective lists
    for x in range(5):
        for i in total_monthly_income[x::5]:
            if x == 0:
                amount.append(i)  
            elif x == 1:
                starting.append(i)
            elif x == 2:
                ending.append(i)
            elif x == 3:
                rate.append(i)
            else:
                bonus.append(i)
    
    total = [0 for x in range(len(Age_List))]
    
    for i in range(len(amount)):        
        first_part = [0 for x in range(starting[i]-starting_age)]
        middle_part = [amount[i]*(12+bonus[i])*(1+rate[i])**x for x in range(ending[i]-starting[i]+1)]
        last_part = [0 for x in range(Final_Age-ending[i])]
        
        total_sub = first_part+middle_part+last_part
        total = [x + y for x, y in zip(total, total_sub)]
    
    retirement_status = []
    
    for age in Age_List:
        if age < ra:
            retirement_status.append(1)
        else:
            retirement_status.append(0)
    
    total = [x * y for x, y in zip(total, retirement_status)]
    
    return total
print(len(income()))
print(income())

38
[0, 3080.0, 3209.36, 3344.1531200000004, 3484.60755104, 3630.9610681836807, 3783.461433047395, 3942.366813235386, 4107.946219391272, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Expenses

In [17]:
def expense():
   
    # Creating lists for different expense streams
    amount = []
    starting = []
    ending = []
    rate = []
    
    # Putting FE values into the respective lists
    for x in range(4):
        for i in total_monthly_expense[x::4]:
            if x == 0:
                amount.append(i)  
            elif x == 1:
                starting.append(i)
            elif x == 2:
                ending.append(i)
            else:
                rate.append(i)
    
    #Calculate Total Expenses
    total = [0 for x in range(len(Age_List))]
    
    for i in range(len(amount)):        
        first_part = [0 for x in range(starting[i]-starting_age)]
        if life_expectancy > ending[i]:
            middle_part = [amount[i]*12*(1+rate[i])**x for x in range(ending[i]-starting[i]+1)]
            last_part = [0 for x in range(Final_Age-ending[i])]
        else:
            middle_part = [amount[i]*12*(1+rate[i])**x for x in range(Final_Age-starting[i]+1)]
            last_part = []
                                                                    
        total_sub = first_part+middle_part+last_part
        total = [x + y for x, y in zip(total, total_sub)]
    
    # Housing Downpayment
    housing_downpayment = []
    
    for idx,age in enumerate(Age_List):
        if age == house_purchase_age:
            price = house_value*(1+housing_price_index)**(idx)*house_downpayment_percentage
            housing_downpayment.append(price)
        else:
            housing_downpayment.append(0)
    
    total = [x + y for x, y in zip(total, housing_downpayment)]

    # Housing Mortgage
    housing_mortgage = []
    
    remaining_value = sum(housing_downpayment)/house_downpayment_percentage - sum(housing_downpayment)
    for idx,age in enumerate(Age_List):
        payment = np.pmt(house_interest_rate,house_loan_years,-remaining_value)
        if age in range(house_purchase_age,house_purchase_age + house_loan_years):  
            housing_mortgage.append(payment)
        else:
            housing_mortgage.append(0)
    
    total = [x + y for x, y in zip(total, housing_mortgage)]
    
    return total

print(len(expense()))
print(expense())

38
[0, 1838.8746049831368, 1608.7506049831366, 1644.4029601831364, 1680.9537547341763, 1718.4256293079022, 1756.8417951208862, 1796.226048312357, 1836.6027846842537, 1877.9970148127215, 1920.4343795404268, 1963.94116585927, 2008.5443231933489, 2054.2714800922454, 2101.1509613449953, 2149.2118055253136, 2004.5091779958398, 2055.022809281335, 2106.809384075224, 2159.9009805539195, 2214.330485263878, 2270.1316134925273, 2327.3389301525385, 2385.9878711923825, 2446.11476554643, 2507.7568576382, 2570.952330450683, 2635.7403291780392, 2702.160985473326, 2770.2554423072534, 2840.065879453396, 2911.6355396156214, 2985.0087552139344, 3060.2309758453252, 3137.348796436627, 3216.4099861068294, 3297.4635177567216, 3380.55959840419]


<ipython-input-17-3e1b4f76c0c6>:53: DeprecationWarning: numpy.pmt is deprecated and will be removed from NumPy 1.20. Use numpy_financial.pmt instead (https://pypi.org/project/numpy-financial/).
  payment = np.pmt(house_interest_rate,house_loan_years,-remaining_value)


## Total

In [4]:
# Calculating Net Assets
def total(inc, exp,IR=investment_rate,IP=percentage_investment):
    ending_balance = []
    start = starting_savings
    for i in range(Final_Age-starting_age+1):
        before = start + inc[i] - exp[i]
        if before < 0:
            ending = 0
        elif before > start:
            ending = start*IP*(1+IR) + start*(1-IP)*(1+bank_interest_rate) + (before-start)*IP*(1+IR)/2 + (before-start)*(1-IP)*(1+bank_interest_rate)/2
        else:
            ending = before*IP*(1+IR) + before*(1-IP)*(1+bank_interest_rate) + (start-before)*IP*(IR)/2 + (start-before)*(1-IP)*(bank_interest_rate)/2
        ending_balance.append(ending)
        start = ending
    
    return ending_balance

total(income(),expense())

<ipython-input-3-3e1b4f76c0c6>:53: DeprecationWarning: numpy.pmt is deprecated and will be removed from NumPy 1.20. Use numpy_financial.pmt instead (https://pypi.org/project/numpy-financial/).
  payment = np.pmt(house_interest_rate,house_loan_years,-remaining_value)


[5185.0,
 6020.368517316243,
 7073.038123773188,
 8216.06099221784,
 9455.249742314467,
 10796.743607837194,
 12247.025403592064,
 13812.93933013756,
 15501.70965624824,
 14162.532953942668,
 12730.584257676619,
 11201.341797782987,
 9570.089051128529,
 7831.906843546332,
 5981.6651426276685,
 4014.014528977361,
 2120.9404687607603,
 106.37453485186875,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

## Solvers

In [5]:
def Calculate_Age():
    for idx,i in enumerate(range(starting_age,Final_Age)):
        x = income()
        y = expense()
        z = total(x,y)
        if z[idx]==0:
            return i # first age when your net assets reach 0 at life expectancy
        elif i == Final_Age-1:
            return "N/A" # no age where your net assets reach 0

In [30]:
def Max_Expense():
    for i in range(0,101):
        x = income()
        y = [amount*(1+i/100) for amount in expense()]
        z = total(x,y)
        if z[life_expectancy-starting_age]>0:
            pass
        elif z[life_expectancy-starting_age] == 0:
            if i == 1:
                return "0%"
            else:
                return str(i+1) + "%" # How much percent you can afford to increase all your expenses
        else:
            return ">100%" # You can return expenses >100%

In [27]:
def Min_Income():
    for i in reversed(range(0,101)):
        x = [amount*i/100 for amount in income()]
        y = expense()
        z = total(x,y)
        if z[life_expectancy-starting_age]>0:
            if i == 0:
                return "100%"
        elif z[life_expectancy-starting_age] == 0:
            return str(100-(i)-1) +"%" # How much percent you can afford to reduce all your income
        else:
            return "0%" # You can't reduce income at all

In [8]:
def Calculate_Retirement():
    # Searches for retirement age from Starting_Age till the end for the first time 
    # Ending_Asset value is positive at Life_Expectancy
    for i in range(starting_age,Final_Age):
        x = income(ra=i)
        y = expense()
        z = total(x,y)
        if z[life_expectancy-starting_age] == 0:
            if i == Final_Age-1:
                return "N/A" # you can't retire
            else:  
                pass
        elif z[life_expectancy-starting_age] > 0:
            return i # the earliest possible retirement age till life expectancy
        else:
            return starting_age # no issues retiring at Starting_Age

In [9]:
def Calculate_Investment_Rate():
    # Searches for percentage from 0-100 till the end for the first time 
    # Ending_Asset value at Life_Expectancy is positive
    for i in range(0,101):
        x = income()
        y = expense()
        z = total(x,y,IR=i/100)
        if z[life_expectancy-starting_age] == 0:
            if i == 100:
                return "N/A" # you would require investment returns of >100%, try tweaking other parameters"
            else:
                pass
        elif z[life_expectancy-starting_age] > 0:
            return str(i) +"%" # the minimum investment rate p.a to reach life expectancy

In [12]:
def Calculate_Investment_Percentage():
    # Searches for percentage allocation from 0-100 till the end for the first time 
    # Ending_Asset value at Life_Expectancy is positive
    for i in range(0,101):
        x = income()
        y = expense()
        z = total(x,y,IP=i/100)
        if z[life_expectancy-starting_age] == 0:
            if i == 100:
                return "N/A" # you would require investment allocation of >100%, try tweaking other parameters
            else:
                pass
        elif z[life_expectancy-starting_age] > 0:
            return str(i) + "%" # the minimum investment percentage to reach life expectancy